In [2]:
import pandas as pd

In [1]:
import psycopg2

# Configurar la conexión
conn = psycopg2.connect(
    dbname="reservas_hoteles",
    user="luisjimse",
    password="admin",
    host="localhost",
    port="5432"
)

# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

# Verificar la conexión
cur.execute("SELECT version();")
print(cur.fetchone())

# Cerrar la conexión
cur.close()
conn.close()

('PostgreSQL 14.13 (Homebrew) on aarch64-apple-darwin23.4.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit',)


In [3]:
df = pd.read_parquet('../data/reservas_hoteles_limpio.parquet')

In [16]:
conn = psycopg2.connect(
    dbname="reservas_hoteles",
    user="luisjimse",
    password="admin",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

df_filtered = df[['id_cliente', 'nombre', 'apellido', 'mail']]

# Convierte las filas en tuplas
records_list = [tuple(row) for row in df_filtered.itertuples(index=False, name=None)]


In [51]:
df.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,HC5311,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-19,2025-03-01,2025-03-02,1,170.00,ibis Styles Madrid Prado,4.700000,Madrid
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,HC8310,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-19,2025-03-01,2025-03-02,2,255.00,Novotel Madrid Center,4.600000,Madrid
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,HP9063,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,3,119.96,Hotel Monte Verde,3.103261,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,HP13646,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,4,211.53,Hotel Brisas del Mar,3.091078,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,HP993,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,5,378.37,Hotel Camino del Sol,3.072519,Madrid


In [21]:
query = """
    INSERT INTO clientes (id_cliente, nombre, apellido, mail)
    VALUES (%s, %s, %s, %s)
    ON CONFLICT (mail) DO NOTHING
"""
cur.executemany(query, records_list)
conn.commit()

In [96]:
conn.rollback()  # Cancela la transacción fallida


In [24]:
query = """
    INSERT INTO ciudad (nombre_ciudad)
    SELECT %s
    WHERE NOT EXISTS (
        SELECT 1 FROM ciudad WHERE nombre_ciudad = %s
    );
"""
cur.execute(query, ('Madrid', 'Madrid'))
conn.commit()


In [60]:
df_hotel = df[['id_hotel','nombre_hotel', 'competencia', 'estrellas']].copy()
df_hotel['id_ciudad'] = id_ciudad  # Agregar id_ciudad correctamente
df_hotel = df_hotel.drop_duplicates(subset=['id_hotel'])


df_hotel['id_hotel'] = df_hotel['id_hotel'].astype(str)  # Asegúrate de que sea un string

records_list = [tuple(row) for row in df_hotel.itertuples(index=False, name=None)]


In [61]:
query = """
    INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s)
"""
cur.executemany(query, records_list)
conn.commit()

In [62]:
df_reservas = df[['id_reserva','fecha_reserva','inicio_estancia','final_estancia','precio_noche','id_cliente','id_hotel']]
df_reservas['fecha_reserva'] = pd.to_datetime(df_reservas['fecha_reserva']).dt.date
df_reservas['inicio_estancia'] = pd.to_datetime(df_reservas['inicio_estancia']).dt.date
df_reservas['final_estancia'] = pd.to_datetime(df_reservas['final_estancia']).dt.date


/var/folders/7h/3lv1slyd33dc0tb3lck5tb2c0000gn/T/ipykernel_27414/3674131598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservas['fecha_reserva'] = pd.to_datetime(df_reservas['fecha_reserva']).dt.date
/var/folders/7h/3lv1slyd33dc0tb3lck5tb2c0000gn/T/ipykernel_27414/3674131598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reservas['inicio_estancia'] = pd.to_datetime(df_reservas['inicio_estancia']).dt.date
/var/folders/7h/3lv1slyd33dc0tb3lck5tb2c0000gn/T/ipykernel_27414/3674131598.py:4: Se

In [80]:

df_reservas = df_reservas.drop_duplicates(subset=['id_reserva'])


In [81]:
query = """
    INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
records_list = [tuple(row) for row in df_reservas.itertuples(index=False, name=None)]
cur.executemany(query, records_list)
conn.commit()


In [82]:
df_eventos = pd.read_parquet('../data/api_eventos.parquet')

In [83]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 62 to 1117
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nombre_evento  29 non-null     object
 1   url_evento     29 non-null     object
 2   codigo_postal  29 non-null     int64 
 3   direccion      17 non-null     object
 4   fecha_inicio   29 non-null     object
 5   fecha_fin      29 non-null     object
 6   localizacion   29 non-null     object
 7   horario        29 non-null     object
dtypes: int64(1), object(7)
memory usage: 2.0+ KB


In [86]:
df_eventos['id_ciudad'] = 1

In [87]:
df_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,fecha_inicio,fecha_fin,localizacion,horario,id_ciudad
62,Asalto a la lectura. Tres cerditos y más,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015,CALLE AMANIEL 29,2025-03-01,2025-03-01,Centro Cultural Clara del Rey - Museo ABC (Cen...,11:00 - 23:59,1
66,Atlantiques + Dahomey,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,2025-03-01,2025-03-01,Cineteca Madrid,19:30 - 23:59,1
81,Baile de Carnaval. Concierto de Origen,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,None,2025-03-01,2025-03-01,,19:30 - 23:59,1
82,Baile de Carnaval. Concierto de Versioneta,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,None,2025-03-01,2025-03-01,,21:15 - 23:59,1
130,Carnaval en Matadero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,2025-03-01,2025-03-01,Matadero Madrid,10:30 - 23:59,1


In [88]:
query = """
    INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT DO NOTHING
"""

df_eventos = df_eventos.rename(columns={'localizacion': 'organizacion'})

records_list = df_eventos[['nombre_evento', 'url_evento', 'codigo_postal', 'direccion', 'horario', 'fecha_inicio', 'fecha_fin', 'organizacion', 'id_ciudad']].values.tolist()

cur.executemany(query, records_list)
conn.commit()


In [92]:
df_eventos_nuevos = pd.read_parquet('../data/eventos_nuevos.parquet')
df_eventos_nuevos['id_ciudad'] = 1

In [95]:

df_eventos_nuevos = df_eventos_nuevos.rename(columns={'localizacion': 'organizacion'})


In [97]:
query = """
    INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT DO NOTHING  -- Ignora eventos duplicados si ya existen
"""

records_list = df_eventos_nuevos[['nombre_evento', 'url_evento', 'codigo_postal', 'direccion', 'horario', 'fecha_inicio', 'fecha_fin', 'organizacion', 'id_ciudad']].values.tolist()

cur.executemany(query, records_list)
conn.commit()
